In [12]:
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras import layers
from keras import models
from keras.applications.resnet50 import ResNet50
from keras.layers import Input, Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D
from keras.layers import Add, Flatten, AveragePooling2D, Dense
from keras.models import Model
from keras import optimizers 
import time

def res_layer(input_layer, n=64):
    L1 = Conv2D(n, (3, 3), padding='same', activation='relu')(input_layer)
    L2 = Conv2D(n, (3, 3), padding='same', activation='relu')(L1)
    L2 = Conv2D(n, (3, 3), padding='same', activation='relu')(L2)
    L3 = Add()([L2, input_layer])
    return L3

In [13]:
main_input = Input(shape=(32,32,3))
L1 = Conv2D(64, (7, 7), strides=(2,2), padding='same', activation='relu')(main_input)
L2 = MaxPooling2D(pool_size=(3,3), strides=(2,2))(L1)

L2 = res_layer(L2)
L2 = res_layer(L2)
L2 = res_layer(L2)

L3 = AveragePooling2D()(L2)
L3 = Flatten()(L3)
L4 = Dense(256)(L3)
L5 = Dense(10,activation='softmax')(L4)

In [14]:
model = Model(main_input, L5)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 16, 16, 64)   9472        input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 7, 7, 64)     0           conv2d_31[0][0]                  
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 7, 7, 64)     36928       max_pooling2d_4[0][0]            
__________________________________________________________________________________________________
conv2d_33 

In [15]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 
x_train = x_train.astype('float32')/255 
x_test = x_test.astype('float32')/255 
y_train = to_categorical(y_train) 
y_test = to_categorical(y_test) 
sgd = optimizers.SGD(lr=0.01, decay=5e-4, momentum=0.9, nesterov=True) 
start_time = time.time()
model.compile(optimizer=sgd, 
              loss='categorical_crossentropy', 
              metrics=['accuracy']) 
end_time = time.time()
history = model.fit(x_train, y_train, epochs=10, shuffle='batch', batch_size=64) 
f = open('log.txt','w') 

Epoch 1/10
50000/50000 [==============================] - 13s 262us/step - loss: 1.5770 - acc: 0.4280
Epoch 2/10
50000/50000 [==============================] - 13s 254us/step - loss: 1.2310 - acc: 0.5630
Epoch 3/10
50000/50000 [==============================] - 13s 255us/step - loss: 1.0676 - acc: 0.6223
Epoch 4/10
50000/50000 [==============================] - 13s 252us/step - loss: 0.9521 - acc: 0.6642
Epoch 5/10
50000/50000 [==============================] - 13s 256us/step - loss: 0.8656 - acc: 0.6955
Epoch 6/10
50000/50000 [==============================] - 13s 254us/step - loss: 0.7977 - acc: 0.7196
Epoch 7/10
50000/50000 [==============================] - 13s 255us/step - loss: 0.7375 - acc: 0.7404
Epoch 8/10
50000/50000 [==============================] - 13s 253us/step - loss: 0.6839 - acc: 0.7587
Epoch 9/10
50000/50000 [==============================] - 13s 254us/step - loss: 0.6384 - acc: 0.7777
Epoch 10/10
50000/50000 [==============================] - 13s 256us/step - loss: 

In [16]:
test_loss, test_acc = model.evaluate(x_test, y_test) 
print(test_loss, test_acc)

10000/10000 [==============================] - 2s 160us/step
0.8045161793708802 0.7298


In [17]:
print(end_time-start_time)

0.023489713668823242
